적대적 탐색 알고리즘의 동작 방식을 이해하고 이를 이용하여 게임 프로그램을 만드는 예를 살펴 봅시다. 여기 제공하는 코드는 GitHub aima-python의 코드를 기반으로 일부 수정한 것임.

In [62]:
from collections import defaultdict
import random
import math
import sys
sys.setrecursionlimit(10000)
infinity = math.inf

## 게임 문제 정의
자신만의 게임 문제를 정의하려면 Game 클래스의 서브 클래스를 정의하면 됨.

In [2]:
class Game:
    """문제를 정의하기 위한 Problem 클래스에 대응되는 게임 정의를 위한 클래스.
    경로 비용과 목표 검사 대신 각 상태에 대한 효용 함수와 종료 검사로 구성됨.
    게임을 정의하려면 이 클래스의 서브 클래스를 만들어서
    actions, result, is_terminal, utility를 구현하면 됨.
    필요에 따라 게임의 초기 상태를 지정하려면,
    클래스 생성자에서 초기 상태를 initial 에 세팅하면 됨."""

    def actions(self, state):
        """주어진 상태에서 허용 가능한 수(move) 리스트"""
        raise NotImplementedError

    def result(self, state, move):
        """주어진 상태(state)에서 수(move)를 두었을 때의 결과 상태 리턴"""
        raise NotImplementedError

    def is_terminal(self, state):
        """state가 종료 상태이면 True 리턴"""
        return not self.actions(state)
    
    def utility(self, state, player):
        """종료 상태 state에서 게임이 종료됐을 때 player의 효용 함수 값"""
        raise NotImplementedError

In [43]:
def play_game(game, strategies: dict, verbose=False):
    """번갈아 가면서 두는 게임 진행 함수.
    strategies: {참가자 이름: 함수} 형태의 딕셔너리. 
    여기서 함수(game, state)는 상태 state에서 참가자의 수(move)를 찾는 함수"""
    #state = game.initial

    state = game.board
    print(strategies)
    while not game.is_terminal():
        player = game.to_move
        move = strategies[player](game, state)
        state = game.result()
        if verbose: 
            print('Player', player, 'move:', move)
            print(state)
    return state

## 적대적 탐색 알고리즘 구현

### 미니맥스 탐색

In [4]:
def minimax_search(game, state):
    """최고의 수를 결정하기 위한 게임 트리 탐색.
    (미니맥스 값, 수) 쌍을 리턴함."""

    player = state.to_move

    def max_value(state):
        if game.is_terminal():
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move

    def min_value(state):
        if game.is_terminal():
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a
        return v, move

    return max_value(state)

### 알파-베타 탐색

In [52]:
def alphabeta_search(game, state):
    """알파-베타 가지치기를 사용하여 최고의 수를 결정하기 위한 게임 트리 탐색."""

    player = game.to_move

    def max_value(state, alpha, beta):
        if game.is_terminal():
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions():
            v2, _ = min_value(game.result(a), alpha, beta)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        if not move:
            game.to_move = '○'
            return min_value(state, alpha, beta)
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal():
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions():
            v2, _ = max_value(game.result(a), alpha, beta)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        if not move:
            game.to_move =  '●'
            return max_value(state, alpha, beta)   
        return v, move

    return max_value(state, -infinity, +infinity)

### 휴리스틱 알파-베타 탐색

In [6]:
def cutoff_depth(d):
    """깊이 d까지만 탐색하도록 하는 중단 함수: depth > d이면 True 리턴."""
    return lambda game, state, depth: depth > d

def h_alphabeta_search(game, state, cutoff=cutoff_depth(6), h=lambda s, p: 0):
    """휴리스틱 알파-베타 탐색"""

    player = state.to_move

    def max_value(state, alpha, beta, depth):
        if game.is_terminal():
            return game.utility(state, player), None
        if cutoff(game, state, depth):
            return h(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta, depth+1)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta, depth):
        if game.is_terminal():
            return game.utility(state, player), None
        if cutoff(game, state, depth):
            return h(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta, depth + 1)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -infinity, +infinity, 0)

## Othello(리버시) 게임

### 게임 정의

In [60]:
class Othello(Game):
    """Othello 게임. 보드 크기: width * height. 모든 판을 채웠을 때 더 많은 색을 가진 사람이 승리
    '●'와 '○'가 게임 플레이. '●'가 먼저 플레이.
    (0, 0) 위치는 보드의 좌상단 끝 위치."""

    def __init__(self, height, width, to_move): # 가로, 세로는 입력받을 수 있도록 수정
        self.height=height
        self.width=width
        self.board = [[ '.' for x in range(width)] for y in range(height)]
        self.board[width//2][height//2]='●'
        self.board[width//2-1][height//2-1]='●'
        self.board[width//2-1][height//2]='○'
        self.board[width//2][height//2-1]='○'
        self.to_move = to_move

    def actions(self):
        move = ((0,1),(1,0),(0,-1),(-1,0),(1,1),(-1,1),(1,-1),(-1,-1))
        """내 돌을 놓을 수 있는 위치들"""
        action = set()
        for i in range(self.width):
            for j in range(self.height):
                for d in move:
                    if(self.feasible(i+d[0], j+d[1], d) > 0):
                        action.add((i,j))
                        break
        return action

    def feasible(self, x, y, dxy): # turn_check가 직관적이지 않아서 가능한 해인가? 라는 뜻의 feasible로 함수명 변경
        if(x >= self.width or y >= self.height or self.board[x][y] == '.'):
            return -1
        elif(self.board[x][y] == self.to_move):
            return 0
        else:
            tmp = self.feasible(x + dxy[0], y + dxy[1], dxy)
            if(tmp != -1):
                return tmp + 1
            return -1
    
    def turn(self, x, y, dxy): # turn_check => turn 
        '''진짜 뒤집음'''
        if(x>=self.width or y>=self.height or self.board[x][y]=='.'):
            return -1
        elif(self.board[x][y]==self.to_move):
            return 0
        else:
            tmp = self.turn(x+dxy[0], y+dxy[1], dxy)
            if(tmp != -1):
                self.board[x][y] = self.to_move
                return 0
            return -1
    
    def check_all_dirt(self,action): # turn_all_dirt => check_all_dirt
        '''8방향 뒤집기 체크'''
        move = ((0,1),(1,0),(0,-1),(-1,0),(1,1),(-1,1),(1,-1),(-1,-1))
        
        for d in move:
            self.turn(action[0]+d[0], action[1]+d[1], d)

    def result(self, action):
        """board의 square 위치에 현재 플레이어의 표시를 위치시킴."""
        board2 = Othello(self.height, self.width, self.to_move)
        board2.board = [self.board[x][:] for x in range(self.height)]
        board2.check_all_dirt(action)
        board2.board[action[0]][action[1]] = self.to_move
        
        if self.to_move=='○': 
            self.to_move = '●'
        else: 
            self.to_move = '○'
        
        # win = k_in_row(board, player, self.k)
        # board.utility = (0 if not win else +1 if player == 'X' else -1)
        return board2.board

    def utility(self, board, player):
        """승리: 1, 패배: -1, 그외: 0"""
        cnt = 0
        for i in range(self.height):
            for j in range(self.width):
                if board[i][j] == '●':
                    cnt += 1
        return cnt
        #return board.utility if player == 'X' else -board.utility

    def is_terminal(self):    # 아래 return문 주석처리 후 내용 추가 => -1이면 단말노드x, 1이면 단말노드o
        """승/패가 결정되었거나 빈 위치가 존재하지 않으면 종료 상태임."""
        for i in range(self.height):
            for j in range(self.width):
                if self.board[i][j] == '.':
                    return 0 # 단말노드가 아님
        return 1 # 단말노드임
                
        # return board.utility != 0 or len(self.squares) == len(board)

    def display(self, board):  # print(board) => 아래 코드로 변경
        for i in range(board.height):
            for j in range(board.width):
                print(board.board[i][j], end=" ")
            print()

# def k_in_row(board, player, square, k):
#     """일직선으로 k개의 player 돌이 놓여 있으면 True 리턴."""
#     def in_row(x, y, dx, dy): return 0 if board[x, y] != player else 1 + in_row(x + dx, y + dy, dx, dy)
#     return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
#                for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))

### 게임 참가자(player) 정의

In [9]:
def random_player(game, state):
    """허용되는 수(move) 중에서 무작위로 하나를 선택하는 플레이어"""
    return random.choice(list(game.actions(state)))

def player(search_algorithm):
    """지정된 탐색 알고리즘을 사용하는 플레이어: (game, state)를 입력 받아 move를 리턴하는 함수."""
    return lambda game, state: search_algorithm(game, state)[1]

In [10]:
def query_player(game, state):
    """다음 수(move)를 직접 입력하는 형태의 플레이어"""
    print("현재 상태:")
    game.display(state)
    print(f"가능한 수: {game.actions(state)}")
    print("")
    move = None
    if game.actions(state):
        move_string = input('당신의 수는? (돌을 둘 위치 입력; 예: (1,1)): ')
        try:
            move = eval(move_string)
        except NameError:
            move = move_string
    else:
        print('가능한 수가 없음: 상대방에게 순서가 넘어감.')
    return move

## 게임하기

In [63]:
play_game(Othello(4, 4, '●'), {'●':player(alphabeta_search), '○':player(minimax_search)}, verbose=True)

{'●': <function player.<locals>.<lambda> at 0x7fb98b85e550>, '○': <function player.<locals>.<lambda> at 0x7fb98b85e790>}


RecursionError: ignored

In [ ]:
play_game(TicTacToe(), {'X':player(h_alphabeta_search), 'O':player(h_alphabeta_search)}, verbose=True).utility

In [ ]:
play_game(TicTacToe(), {'X':player(h_alphabeta_search), 'O':random_player}, verbose=True).utility

In [ ]:
%%time
play_game(TicTacToe(), {'X':player(minimax_search), 'O':player(h_alphabeta_search)})

In [ ]:
%%time
play_game(TicTacToe(), {'X':player(alphabeta_search), 'O':player(h_alphabeta_search)})

In [ ]:
%%time
play_game(TicTacToe(), {'X':player(h_alphabeta_search), 'O':player(h_alphabeta_search)})

In [ ]:
play_game(TicTacToe(), {'●':query_player, '○':random_player})